In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

from pyspark.ml import Pipeline
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit
from pyspark.ml.evaluation import BinaryClassificationEvaluator, RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.regression import DecisionTreeRegressor

In [0]:
IS_SPARK_SUBMIT_CLI = True

if IS_SPARK_SUBMIT_CLI:
    sc = SparkContext.getOrCreate()
    spark = SparkSession(sc)

#LOAD DATA  -- KTSample.csv

In [0]:
file_location = "/FileStore/tables/KTSample.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df1 = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

In [0]:
temp_table_name1 = "KTSample_csv"

df1.createOrReplaceTempView(temp_table_name1)

In [0]:
if IS_SPARK_SUBMIT_CLI:
    KTSample = spark.read.csv('/FileStore/tables/KTSample.csv', inferSchema=True, header=True)
else:
    KTSample = sqlContext.sql("select * from KTSample_csv")
    
KTSample.show(5)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
 1| 2019-11-01 00:30:41| 2019-11-01 00:32:25| 1| 0.0| 1| N| 145| 145| 2| 3.0| 0.5| 0.5| 0.0| 0.0| 0.3| 4.3| 0.0|
 1| 2019-11-01 00:34:01| 2019-11-01 00:34:09| 1| 0.0| 1| N| 145| 145| 2| 2.5| 0.5| 0.5| 0.0| 0.0| 0.3| 3.8| 0.0|
 2| 2019-11-01 00:41:59| 2019-11-01 00:42:23| 1| 0.0| 1| N| 193| 193| 1| 2.5| 0.5| 0.5| 0.95| 0.0| 0.3| 4.75| 0.0|
 2| 2019-11-01 00:02:39| 2019-11-01 00:02:51| 1| 0.0| 1| N| 193| 193| 1| 2.5| 0.5| 0.5| 0.95| 0.0| 0.3| 4.75| 0.0|
 2| 2019-11-01 00:18:30| 2019-11-01 00:18:39| 2| 0.0| 1| N| 226| 226| 2| 2.5| 0.0| 0.5| 0.0| 0.0| 0.3| 3.3| 0.0|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
only showing top 5 rows

#Select Data and Calculate the Trip Time in SEC

In [0]:
timediff=KTSample.select('tpep_pickup_datetime', 'tpep_dropoff_datetime','passenger_count','trip_distance',col('fare_amount').alias('label'))

timediff.show(5)

+--------------------+---------------------+---------------+-------------+-----+
tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|label|
+--------------------+---------------------+---------------+-------------+-----+
 2019-11-01 00:30:41| 2019-11-01 00:32:25| 1| 0.0| 3.0|
 2019-11-01 00:34:01| 2019-11-01 00:34:09| 1| 0.0| 2.5|
 2019-11-01 00:41:59| 2019-11-01 00:42:23| 1| 0.0| 2.5|
 2019-11-01 00:02:39| 2019-11-01 00:02:51| 1| 0.0| 2.5|
 2019-11-01 00:18:30| 2019-11-01 00:18:39| 2| 0.0| 2.5|
+--------------------+---------------------+---------------+-------------+-----+
only showing top 5 rows

In [0]:
df2=timediff.withColumn('tpep_pickup_datetime',to_timestamp(col('tpep_pickup_datetime')))\
  .withColumn('tpep_dropoff_datetime', to_timestamp(col('tpep_dropoff_datetime')))\
  .withColumn('trip_time_in_secs',col("tpep_dropoff_datetime").cast("long") - col('tpep_pickup_datetime').cast("long"))
df2.show(5)

+--------------------+---------------------+---------------+-------------+-----+-----------------+
tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|label|trip_time_in_secs|
+--------------------+---------------------+---------------+-------------+-----+-----------------+
 2019-11-01 00:30:41| 2019-11-01 00:32:25| 1| 0.0| 3.0| 104|
 2019-11-01 00:34:01| 2019-11-01 00:34:09| 1| 0.0| 2.5| 8|
 2019-11-01 00:41:59| 2019-11-01 00:42:23| 1| 0.0| 2.5| 24|
 2019-11-01 00:02:39| 2019-11-01 00:02:51| 1| 0.0| 2.5| 12|
 2019-11-01 00:18:30| 2019-11-01 00:18:39| 2| 0.0| 2.5| 9|
+--------------------+---------------------+---------------+-------------+-----+-----------------+
only showing top 5 rows

In [0]:
data=df2.select('passenger_count','trip_distance','trip_time_in_secs','label')

data.show(5)

+---------------+-------------+-----------------+-----+
passenger_count|trip_distance|trip_time_in_secs|label|
+---------------+-------------+-----------------+-----+
 1| 0.0| 104| 3.0|
 1| 0.0| 8| 2.5|
 1| 0.0| 24| 2.5|
 1| 0.0| 12| 2.5|
 2| 0.0| 9| 2.5|
+---------------+-------------+-----------------+-----+
only showing top 5 rows

#Setup Train and Test datasets

In [0]:
splits = data.randomSplit([0.7, 0.3])
train = splits[0]
test = splits[1].withColumnRenamed("label", "trueLabel")

#Setup GBT-Regression

In [0]:
assembler = VectorAssembler(inputCols = ['passenger_count', 'trip_time_in_secs', 'trip_distance'], outputCol="features")
gbt = GBTRegressor(labelCol="label")

In [0]:
paramGrid = ParamGridBuilder()\
  .addGrid(gbt.maxDepth, [2, 3])\
  .addGrid(gbt.maxIter, [10, 20])\
  .build()
  
#evaluator = RegressionEvaluator(metricName="rmse", labelCol=gbt.getLabelCol(), predictionCol=gbt.getPredictionCol())

cv = CrossValidator(estimator=gbt, evaluator=RegressionEvaluator(), estimatorParamMaps=paramGrid)

In [0]:
pipeline = Pipeline(stages=[assembler, cv])
pipelineModel = pipeline.fit(train)

/databricks/spark/python/pyspark/ml/util.py:800: UserWarning: Cannot find mlflow module. To enable MLflow logging, install mlflow from PyPI.
 warnings.warn(_MLflowInstrumentation._NO_MLFLOW_WARNING)

In [0]:
predictions = pipelineModel.transform(test)

In [0]:
predicted = predictions.select("features", "prediction", "trueLabel")
predicted.show(10)

+---------------+-----------------+---------+
 features| prediction|trueLabel|
+---------------+-----------------+---------+
 (3,[],[])| 13.9535994897476| 30.0|
 [0.0,2.0,0.0]| 13.9535994897476| 52.0|
 [0.0,3.0,0.0]| 13.9535994897476| 9.3|
 [0.0,8.0,0.0]| 13.9535994897476| 2.5|
 [0.0,9.0,0.0]| 13.9535994897476| 52.0|
 [0.0,4.0,0.1]| 13.9535994897476| 14.0|
 [0.0,94.0,0.1]| 13.9535994897476| 3.0|
[0.0,124.0,0.2]| 13.9535994897476| 3.5|
[0.0,131.0,0.2]| 13.9535994897476| 3.5|
[0.0,165.0,0.3]|4.965279479032191| 4.0|
+---------------+-----------------+---------+
only showing top 10 rows

In [0]:
predicted.createOrReplaceTempView("regressionPredictions")

In [0]:
dataPred = spark.sql("SELECT trueLabel, prediction FROM regressionPredictions")

dataPred.show(10)

+---------+-----------------+
trueLabel| prediction|
+---------+-----------------+
 30.0| 13.9535994897476|
 52.0| 13.9535994897476|
 9.3| 13.9535994897476|
 2.5| 13.9535994897476|
 52.0| 13.9535994897476|
 14.0| 13.9535994897476|
 3.0| 13.9535994897476|
 3.5| 13.9535994897476|
 3.5| 13.9535994897476|
 4.0|4.965279479032191|
+---------+-----------------+
only showing top 10 rows

#RMSE/R2 for GBT-Regression

In [0]:
evaluator  = RegressionEvaluator(labelCol="trueLabel", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print( "Root Mean Square Error (RMSE) for GBT Regression :", rmse)

Root Mean Square Error (RMSE) for GBT Regression : 5.5503173808683615

In [0]:
evaluator  = RegressionEvaluator(labelCol="trueLabel", predictionCol="prediction", metricName="r2")
r2 = evaluator.evaluate(predictions)
print( "Coefficient of Determination (R2) for GBT Regression :", r2)

Coefficient of Determination (R2) for GBT Regression : 0.7651795550483502

#Setup Linear Regression

In [0]:
assembler = VectorAssembler(inputCols = ['passenger_count', 'trip_time_in_secs', 'trip_distance'], outputCol="features")
lr = LinearRegression(labelCol="label",featuresCol="features", maxIter=10, regParam=0.3)
pipeline1 = Pipeline(stages=[assembler, lr])

In [0]:
paramGrid1 = ParamGridBuilder().addGrid(lr.regParam, [0.3, 0.01]).addGrid(lr.maxIter, [10, 5]).build()
trainval = TrainValidationSplit(estimator=pipeline1, evaluator=RegressionEvaluator(), estimatorParamMaps=paramGrid1, trainRatio=0.8)

In [0]:
pipelineModel = trainval.fit(train)

In [0]:
predictions = pipelineModel.transform(test)

In [0]:
predicted = predictions.select("features", "prediction", "trueLabel")
predicted.show(10)

+---------------+-----------------+---------+
 features| prediction|trueLabel|
+---------------+-----------------+---------+
 (3,[],[])| 5.22390440170772| 30.0|
 [0.0,2.0,0.0]|5.224017290745073| 52.0|
 [0.0,3.0,0.0]| 5.22407373526375| 9.3|
 [0.0,8.0,0.0]|5.224355957857134| 2.5|
 [0.0,9.0,0.0]|5.224412402375811| 52.0|
 [0.0,4.0,0.1]|5.492851888228252| 14.0|
 [0.0,94.0,0.1]|5.497931894909164| 3.0|
[0.0,124.0,0.2]|5.768346938915292| 3.5|
[0.0,131.0,0.2]| 5.76874205054603| 3.5|
[0.0,165.0,0.3]|6.039382872626866| 4.0|
+---------------+-----------------+---------+
only showing top 10 rows

In [0]:
predicted.createOrReplaceTempView("regressionPredictions")

In [0]:
dataPred = spark.sql("SELECT trueLabel, prediction FROM regressionPredictions")

dataPred.show(10)

+---------+-----------------+
trueLabel| prediction|
+---------+-----------------+
 30.0| 5.22390440170772|
 52.0|5.224017290745073|
 9.3| 5.22407373526375|
 2.5|5.224355957857134|
 52.0|5.224412402375811|
 14.0|5.492851888228252|
 3.0|5.497931894909164|
 3.5|5.768346938915292|
 3.5| 5.76874205054603|
 4.0|6.039382872626866|
+---------+-----------------+
only showing top 10 rows

#RMSE/R2 for Linear Regression

In [0]:
evaluator  = RegressionEvaluator(labelCol="trueLabel", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print ("Root Mean Square Error (RMSE) for Linear Regression :", rmse)

Root Mean Square Error (RMSE) for Linear Regression : 5.800560237034428

In [0]:
evaluator  = RegressionEvaluator(labelCol="trueLabel", predictionCol="prediction", metricName="r2")
r2 = evaluator.evaluate(predictions)
print( "Coefficient of Determination (R2) for Linear Regression :", r2)

Coefficient of Determination (R2) for Linear Regression : 0.7435278849894993

#Decision Forest Regression

In [0]:
assembler = VectorAssembler(inputCols = ['passenger_count', 'trip_time_in_secs', 'trip_distance'], outputCol="features")
dt = DecisionTreeRegressor(labelCol="label",featuresCol="features")

In [0]:
paramGrid2 = ParamGridBuilder()\
  .addGrid(dt.maxDepth, [2,3])\
  .addGrid(dt.maxBins, [10,20])\
  .build()

In [0]:
#evaluator = RegressionEvaluator(metricName="rmse", labelCol=dt.getLabelCol(), predictionCol=dt.getPredictionCol())

dtcv = CrossValidator(estimator = dt, estimatorParamMaps = paramGrid2, evaluator = RegressionEvaluator(), numFolds=2)

In [0]:
pipeline2 = Pipeline(stages=[assembler, dtcv])
pipelineModel = pipeline2.fit(train)

In [0]:
predictions = pipelineModel.transform(test)

In [0]:
predicted = predictions.select("features","prediction","truelabel")
predicted.show(10)

+---------------+-----------------+---------+
 features| prediction|truelabel|
+---------------+-----------------+---------+
 (3,[],[])|6.631192881192881| 30.0|
 [0.0,2.0,0.0]|6.631192881192881| 52.0|
 [0.0,3.0,0.0]|6.631192881192881| 9.3|
 [0.0,8.0,0.0]|6.631192881192881| 2.5|
 [0.0,9.0,0.0]|6.631192881192881| 52.0|
 [0.0,4.0,0.1]|6.631192881192881| 14.0|
 [0.0,94.0,0.1]|6.631192881192881| 3.0|
[0.0,124.0,0.2]|6.631192881192881| 3.5|
[0.0,131.0,0.2]|6.631192881192881| 3.5|
[0.0,165.0,0.3]|6.631192881192881| 4.0|
+---------------+-----------------+---------+
only showing top 10 rows

In [0]:
predicted.createOrReplaceTempView("regressionPredictions")

In [0]:
dataPred = spark.sql("SELECT trueLabel, prediction FROM regressionPredictions")

dataPred.show(5)

+---------+-----------------+
trueLabel| prediction|
+---------+-----------------+
 30.0|6.631192881192881|
 52.0|6.631192881192881|
 9.3|6.631192881192881|
 2.5|6.631192881192881|
 52.0|6.631192881192881|
+---------+-----------------+
only showing top 5 rows

#RMSE for Decision Forest Regression

In [0]:
evaluator  = RegressionEvaluator(labelCol="trueLabel", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print( "Root Mean Square Error (RMSE) for Decision Forest Regression :", rmse)

Root Mean Square Error (RMSE) for Decision Forest Regression : 6.204744251541987

In [0]:
evaluator  = RegressionEvaluator(labelCol="trueLabel", predictionCol="prediction", metricName="r2")
r2 = evaluator.evaluate(predictions)
print( "Coefficient of Determination (R2) for Decision Forest Regression :", r2)

Coefficient of Determination (R2) for Decision Forest Regression : 0.7065405889317031